# Lab 2, Training

In [1]:
from typing import Dict, Tuple
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import models, transforms
from torchvision.utils import save_image, make_grid
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import numpy as np
from IPython.display import HTML
from diffusion_utilities import *

# Setting Things Up

In [2]:
class ContextUnet(nn.Module):
    def __init__(self, in_channels, n_feat=256, n_cfeat=10, height=28):  # cfeat - context features
        super(ContextUnet, self).__init__()

        # number of input channels, number of intermediate feature maps and number of classes
        self.in_channels = in_channels
        self.n_feat = n_feat
        self.n_cfeat = n_cfeat
        self.h = height  #assume h == w. must be divisible by 4, so 28,24,20,16...

        # Initialize the initial convolutional layer
        self.init_conv = ResidualConvBlock(in_channels, n_feat, is_res=True)

        # Initialize the down-sampling path of the U-Net with two levels
        self.down1 = UnetDown(n_feat, n_feat)        # down1 #[10, 256, 8, 8]
        self.down2 = UnetDown(n_feat, 2 * n_feat)    # down2 #[10, 256, 4,  4]

         # original: self.to_vec = nn.Sequential(nn.AvgPool2d(7), nn.GELU())
        self.to_vec = nn.Sequential(nn.AvgPool2d((4)), nn.GELU())

        # Embed the timestep and context labels with a one-layer fully connected neural network
        self.timeembed1 = EmbedFC(1, 2*n_feat)
        self.timeembed2 = EmbedFC(1, 1*n_feat)
        self.contextembed1 = EmbedFC(n_cfeat, 2*n_feat)
        self.contextembed2 = EmbedFC(n_cfeat, 1*n_feat)

        # Initialize the up-sampling path of the U-Net with three levels
        self.up0 = nn.Sequential(
            nn.ConvTranspose2d(2 * n_feat, 2 * n_feat, self.h//4, self.h//4), # up-sample
            nn.GroupNorm(8, 2 * n_feat), # normalize
            nn.ReLU(),
        )
        self.up1 = UnetUp(4 * n_feat, n_feat)
        self.up2 = UnetUp(2 * n_feat, n_feat)

        # Initialize the final convolutional layers to map to the same number of channels as the input image
        self.out = nn.Sequential(
            nn.Conv2d(2 * n_feat, n_feat, 3, 1, 1), # reduce number of feature maps   #in_channels, out_channels, kernel_size, stride=1, padding=0
            nn.GroupNorm(8, n_feat), # normalize
            nn.ReLU(),
            nn.Conv2d(n_feat, self.in_channels, 3, 1, 1), # map to same number of channels as input
        )

    def forward(self, x, t, c=None):
        """
        x : (batch, n_feat, h, w) : input image
        t : (batch, n_cfeat)      : time step
        c : (batch, n_classes)    : context label
        """
        # x is the input image, c is the context label, t is the timestep, context_mask says which samples to block the context on

        # pass the input image through the initial convolutional layer
        x = self.init_conv(x)
        # pass the result through the down-sampling path
        down1 = self.down1(x)       #[10, 256, 8, 8]
        down2 = self.down2(down1)   #[10, 256, 4, 4]

        # convert the feature maps to a vector and apply an activation
        hiddenvec = self.to_vec(down2)

        # mask out context if context_mask == 1
        if c is None:
            c = torch.zeros(x.shape[0], self.n_cfeat).to(x)

        # embed context and timestep
        cemb1 = self.contextembed1(c).view(-1, self.n_feat * 2, 1, 1)     # (batch, 2*n_feat, 1,1)
        temb1 = self.timeembed1(t).view(-1, self.n_feat * 2, 1, 1)
        cemb2 = self.contextembed2(c).view(-1, self.n_feat, 1, 1)
        temb2 = self.timeembed2(t).view(-1, self.n_feat, 1, 1)
        #print(f"uunet forward: cemb1 {cemb1.shape}. temb1 {temb1.shape}, cemb2 {cemb2.shape}. temb2 {temb2.shape}")


        up1 = self.up0(hiddenvec)
        up2 = self.up1(cemb1*up1 + temb1, down2)  # add and multiply embeddings
        up3 = self.up2(cemb2*up2 + temb2, down1)
        out = self.out(torch.cat((up3, x), 1))
        return out


In [3]:
# hyperparameters

# diffusion hyperparameters
timesteps = 500
beta1 = 1e-4
beta2 = 0.02

# network hyperparameters
device = torch.device("cuda:0" if torch.cuda.is_available() else torch.device('cpu'))
n_feat = 64 # 64 hidden dimension feature
n_cfeat = 5 # context vector is of size 5
height = 16 # 16x16 image
save_dir = './weights/'

# training hyperparameters
batch_size = 100
n_epoch = 32
lrate=1e-3

In [4]:
# construct DDPM noise schedule
b_t = (beta2 - beta1) * torch.linspace(0, 1, timesteps + 1, device=device) + beta1
a_t = 1 - b_t
ab_t = torch.cumsum(a_t.log(), dim=0).exp()
ab_t[0] = 1

In [5]:
# construct model
nn_model = ContextUnet(in_channels=3, n_feat=n_feat, n_cfeat=n_cfeat, height=height).to(device)

# Training

In [6]:
# load dataset and construct optimizer
dataset = CustomDataset("./sprites_1788_16x16.npy", "./sprite_labels_nc_1788_16x16.npy", transform, null_context=False)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=1)
optim = torch.optim.Adam(nn_model.parameters(), lr=lrate)

sprite shape: (89400, 16, 16, 3)
labels shape: (89400, 5)


In [7]:
# helper function: perturbs an image to a specified noise level
def perturb_input(x, t, noise):
    return ab_t.sqrt()[t, None, None, None] * x + (1 - ab_t[t, None, None, None]).sqrt() * noise

In [8]:
import torch
import numpy as np
import matplotlib.pyplot as plt

def compute_noise_schedule_and_distribution(beta1=1e-4, beta2=2e-2, timesteps=1000):
    """Compute the exact noise schedule and x_t distribution"""

    # 1. Beta schedule
    b_t = (beta2 - beta1) * torch.linspace(0, 1, timesteps + 1) + beta1

    # 2. Alpha values
    a_t = 1 - b_t

    # 3. Cumulative alpha (the way it's computed in the code)
    ab_t = torch.cumsum(a_t.log(), dim=0).exp()
    ab_t[0] = 1  # Override first value

    # 4. Alternative computation (should be equivalent)
    ab_t_alternative = torch.cumprod(a_t, dim=0)

    print("=== Noise Schedule Analysis ===")
    print(f"Beta range: [{beta1:.6f}, {beta2:.6f}]")
    print(f"Timesteps: {timesteps}")

    # Show key values
    key_timesteps = [0, 1, 50, 100, 250, 500, 750, 999, 1000]
    print(f"\n{'t':>4} | {'β_t':>8} | {'α_t':>8} | {'ᾱ_t':>8} | {'√ᾱ_t':>8} | {'√(1-ᾱ_t)':>10}")
    print("-" * 65)

    for t in key_timesteps:
        if t < len(b_t):
            beta_val = b_t[t].item()
            alpha_val = a_t[t].item()
            alpha_bar_val = ab_t[t].item()
            sqrt_alpha_bar = alpha_bar_val**0.5
            sqrt_one_minus_alpha_bar = (1 - alpha_bar_val)**0.5

            print(f"{t:4d} | {beta_val:8.6f} | {alpha_val:8.6f} | {alpha_bar_val:8.6f} | {sqrt_alpha_bar:8.6f} | {sqrt_one_minus_alpha_bar:10.6f}")

    return b_t, a_t, ab_t

# Compute the schedule
b_t, a_t, ab_t = compute_noise_schedule_and_distribution()

=== Noise Schedule Analysis ===
Beta range: [0.000100, 0.020000]
Timesteps: 1000

   t |      β_t |      α_t |      ᾱ_t |     √ᾱ_t |   √(1-ᾱ_t)
-----------------------------------------------------------------
   0 | 0.000100 | 0.999900 | 1.000000 | 1.000000 |   0.000000
   1 | 0.000120 | 0.999880 | 0.999780 | 0.999890 |   0.014830
  50 | 0.001095 | 0.998905 | 0.969976 | 0.984874 |   0.173274
 100 | 0.002090 | 0.997910 | 0.895232 | 0.946167 |   0.323679
 250 | 0.005075 | 0.994925 | 0.521750 | 0.722323 |   0.691556
 500 | 0.010050 | 0.989950 | 0.077992 | 0.279271 |   0.960212
 750 | 0.015025 | 0.984975 | 0.003319 | 0.057610 |   0.998339
 999 | 0.019980 | 0.980020 | 0.000041 | 0.006385 |   0.999980
1000 | 0.020000 | 0.980000 | 0.000040 | 0.006321 |   0.999980


In [13]:
def analyze_xt_distribution(x0_mean=0.5, x0_std=0.3, timesteps_to_analyze=[0, 100, 500, 1000]):
    """Analyze the distribution of x_t for different timesteps"""

    print("\n=== X_t Distribution Analysis ===")
    print("Assuming image x₀ ~ N(0.5, 0.3²) for illustration")
    print(f"{'t':>4} | {'E[x_t]':>8} | {'Var[x_t]':>10} | {'Std[x_t]':>10} | Signal Strength")
    print("-" * 70)

    for t in timesteps_to_analyze:
        if t < len(ab_t):
            # Coefficients from the perturb_input function
            signal_coeff = ab_t[t].sqrt().item()      # √ᾱ_t
            noise_coeff = (1 - ab_t[t]).sqrt().item() # √(1-ᾱ_t)

            # x_t = signal_coeff * x₀ + noise_coeff * ε
            # where x₀ ~ N(x0_mean, x0_std²) and ε ~ N(0, 1)

            # Mean: E[x_t] = signal_coeff * E[x₀] + noise_coeff * E[ε]
            mean_xt = signal_coeff * x0_mean + noise_coeff * 0

            # Variance: Var[x_t] = signal_coeff² * Var[x₀] + noise_coeff² * Var[ε]
            var_xt = (signal_coeff**2) * (x0_std**2) + (noise_coeff**2) * 1
            std_xt = var_xt**0.5

            # Signal-to-noise ratio
            signal_strength = signal_coeff / noise_coeff if noise_coeff > 0 else float('inf')

            print(f"{t:4d} | {mean_xt:8.4f} | {var_xt:10.6f} | {std_xt:10.6f} | {signal_strength:12.4f}")

analyze_xt_distribution()


=== X_t Distribution Analysis ===
Assuming image x₀ ~ N(0.5, 0.3²) for illustration
   t |   E[x_t] |   Var[x_t] |   Std[x_t] | Signal Strength
----------------------------------------------------------------------
   0 |   0.5000 |   0.090000 |   0.300000 |          inf
 100 |   0.4731 |   0.185339 |   0.430510 |       2.9232
 500 |   0.1396 |   0.929027 |   0.963860 |       0.2908
1000 |   0.0032 |   0.999964 |   0.999982 |       0.0063


#### This code will take hours to run on a CPU. We recommend you skip this step here and check the intermediate results below.
If you decide to try it, you could download to your own machine. Be sure to change the cell type. 
Note, the CPU run time in the course is limited so you will not be able to fully train the network using the class platform.

In [9]:
# # training without context code

# # set into train mode
# nn_model.train()

# for ep in range(n_epoch):
#     print(f'epoch {ep}')

#     # linearly decay learning rate
#     optim.param_groups[0]['lr'] = lrate*(1-ep/n_epoch)

#     pbar = tqdm(dataloader, mininterval=2 )
#     for x, _ in pbar:   # x: images
#         optim.zero_grad()
#         x = x.to(device)

#         # perturb data
#         noise = torch.randn_like(x)
#         t = torch.randint(1, timesteps + 1, (x.shape[0],)).to(device)
#         x_pert = perturb_input(x, t, noise)

#         # use network to recover noise
#         pred_noise = nn_model(x_pert, t / timesteps)

#         # loss is mean squared error between the predicted and true noise
#         loss = F.mse_loss(pred_noise, noise)
#         loss.backward()

#         optim.step()

#     # save model periodically
#     if ep%4==0 or ep == int(n_epoch-1):
#         if not os.path.exists(save_dir):
#             os.mkdir(save_dir)
#         torch.save(nn_model.state_dict(), save_dir + f"model_{ep}.pth")
#         print('saved model at ' + save_dir + f"model_{ep}.pth")

# Sampling

In [10]:
# helper function; removes the predicted noise (but adds some noise back in to avoid collapse)
def denoise_add_noise(x, t, pred_noise, z=None):
    if z is None:
        z = torch.randn_like(x)
    noise = b_t.sqrt()[t] * z
    mean = (x - pred_noise * ((1 - a_t[t]) / (1 - ab_t[t]).sqrt())) / a_t[t].sqrt()
    return mean + noise

In [11]:
# sample using standard algorithm
@torch.no_grad()
def sample_ddpm(n_sample, save_rate=20):
    # x_T ~ N(0, 1), sample initial noise
    samples = torch.randn(n_sample, 3, height, height).to(device)

    # array to keep track of generated steps for plotting
    intermediate = []
    for i in range(timesteps, 0, -1):
        print(f'sampling timestep {i:3d}', end='\r')

        # reshape time tensor
        t = torch.tensor([i / timesteps])[:, None, None, None].to(device)

        # sample some random noise to inject back in. For i = 1, don't add back in noise
        z = torch.randn_like(samples) if i > 1 else 0

        eps = nn_model(samples, t)    # predict noise e_(x_t,t)
        samples = denoise_add_noise(samples, i, eps, z)
        if i % save_rate ==0 or i==timesteps or i<8:
            intermediate.append(samples.detach().cpu().numpy())

    intermediate = np.stack(intermediate)
    return samples, intermediate

#### View Epoch 0 

In [12]:
# load in model weights and set to eval mode
nn_model.load_state_dict(torch.load(f"{save_dir}/model_0.pth", map_location=device))
nn_model.eval()
print("Loaded in Model")

FileNotFoundError: [Errno 2] No such file or directory: './weights//model_0.pth'

In [ ]:
# visualize samples
plt.clf()
samples, intermediate_ddpm = sample_ddpm(32)
animation_ddpm = plot_sample(intermediate_ddpm,32,4,save_dir, "ani_run", None, save=False)
HTML(animation_ddpm.to_jshtml())

#### View Epoch 4 

In [ ]:
# load in model weights and set to eval mode
nn_model.load_state_dict(torch.load(f"{save_dir}/model_4.pth", map_location=device))
nn_model.eval()
print("Loaded in Model")

In [ ]:
# visualize samples
plt.clf()
samples, intermediate_ddpm = sample_ddpm(32)
animation_ddpm = plot_sample(intermediate_ddpm,32,4,save_dir, "ani_run", None, save=False)
HTML(animation_ddpm.to_jshtml())

#### View Epoch 8

In [ ]:
# load in model weights and set to eval mode
nn_model.load_state_dict(torch.load(f"{save_dir}/model_8.pth", map_location=device))
nn_model.eval()
print("Loaded in Model")

In [ ]:
# visualize samples
plt.clf()
samples, intermediate_ddpm = sample_ddpm(32)
animation_ddpm = plot_sample(intermediate_ddpm,32,4,save_dir, "ani_run", None, save=False)
HTML(animation_ddpm.to_jshtml())

#### View Epoch 31 

In [ ]:
# load in model weights and set to eval mode
nn_model.load_state_dict(torch.load(f"{save_dir}/model_31.pth", map_location=device))
nn_model.eval()
print("Loaded in Model")

In [ ]:
# visualize samples
plt.clf()
samples, intermediate_ddpm = sample_ddpm(32)
animation_ddpm = plot_sample(intermediate_ddpm,32,4,save_dir, "ani_run", None, save=False)
HTML(animation_ddpm.to_jshtml())

# Acknowledgments
Sprites by ElvGames, [FrootsnVeggies](https://zrghr.itch.io/froots-and-veggies-culinary-pixels) and  [kyrise](https://kyrise.itch.io/)   
This code is modified from, https://github.com/cloneofsimo/minDiffusion   
Diffusion model is based on [Denoising Diffusion Probabilistic Models](https://arxiv.org/abs/2006.11239) and [Denoising Diffusion Implicit Models](https://arxiv.org/abs/2010.02502)
